In [12]:
import pandas as panda
import ortools.sat.python as OR
import matplotlib as plot

In [15]:
mock_data = panda.read_csv('MOCK_DATA.csv')

In [16]:
mock_data

,activity_name,category,involved_person,estimated_durations,activity_description,to_do_list,start_date,end_date,preferred_time_start,preferred_time_end,status,frequency
0,Activity Nueea 658,Recurring medium activity,Dotti,114,This activity involves jy8s tasks for ic7 hours.,"Task-408 Qfii, Task-482 Eddy, Task-686 Eaei",20/05/2025,28/03/2025,1:04,02:58,Scheduled,Daily
1,Activity Vxeon 241,Recurring medium activity,Flory,321,This activity involves eu5f tasks for yv1 hours.,"Task-355 Dacc, Task-377 Lalq, Task-951 Fqpl",03/05/2025,26/09/2025,3:55,09:16,Scheduled,Monthly
2,Activity Nlenv 770,Bucket list activity,Martelle,171,This activity involves lf5m tasks for iy9 hours.,"Task-515 Prwj, Task-446 Hpev, Task-661 Pzaz",19/08/2025,03/01/2025,1:33,04:24,Scheduled,Monthly
3,Activity Zycef 056,Locked In activity,Brittne,228,This activity involves as4b tasks for li1 hours.,"Task-782 Icky, Task-158 Ewxx, Task-655 Icoi",01/01/2025,25/11/2025,15:37,19:25,Completed,Weekly
4,Activity Bqfcq 149,Recurring medium activity,Mildrid,453,This activity involves cm4q tasks for pf3 hours.,"Task-638 Rkbc, Task-890 Vset, Task-801 Zlwa",23/05/2025,10/07/2025,19:52,03:25,Completed,Weekly
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Activity Wtvbv 822,Recurring medium activity,Dev,168,This activity involves Pe0t tasks for lk9 hours.,"Task-524 Xazt, Task-839 Idbg, Task-608 Ochy",24/09/2025,19/11/2025,10:10,12:58,Scheduled,Daily
996,Activity Esyjx 625,Locked In activity,Felita,167,This activity involves 2m5c tasks for pf9 hours.,"Task-533 Cukk, Task-005 Bxjh, Task-429 Dfko",02/01/2025,12/02/2025,12:49,15:36,Scheduled,One-off
997,Activity Ntbsy 831,Recurring low activity,Amerigo,339,This activity involves Fq9v tasks for th8 hours.,"Task-039 Huwx, Task-737 Nxpw, Task-667 Cmir",25/11/2025,15/08/2025,8:01,13:40,Completed,Daily
998,Activity Vzkol 330,Locked In activity,Natalya,355,This activity involves 8z4h tasks for pl7 hours.,"Task-390 Lahs, Task-671 Hmed, Task-580 Lplw",15/02/2025,23/10/2025,4:11,10:06,Completed,Weekly


In [ ]:
mock_data.drop(columns = ["involved_person", "activity_duration", "to_do_list"])